# [ 2-3. 미결계정 DB에 대한 CRUD 기능 구현 ]

## 1. CRUD 기능 구현 모듈 작성
### 1-1. 프롬프트 작성
![프롬프트](image/CH02-03-01.png)

### 1-2. ChatGPT 답변

![ChatGPT 답변](image/CH02-03-02.png)
![ChatGPT 답변](image/CH02-03-03.png)
![ChatGPT 답변](image/CH02-03-04.png)
![ChatGPT 답변](image/CH02-03-05.png)

### 1-3. ChatGPT 작성 코드

```
# transaction_manager.py
from connector import Connector

class TransactionManager:
    def create_transaction(
            self, account_name, client_name, 
            transaction_date, amount_receivable,
            amount_payable, settlement_status, notes
        ):
        with Connector() as db:
            query = """
                INSERT INTO transaction_records 
                    (account_name, client_name, 
                    transaction_date, amount_receivable,
                    amount_payable, settlement_status, 
                    notes) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            db.cursor.execute(
                query, 
                (account_name, client_name, 
                transaction_date, amount_receivable,
                amount_payable, settlement_status, notes)
            )
            db.connection.commit()
            return db.cursor.lastrowid

    def read_transaction(self, transaction_id):
        with Connector() as db:
            query = """
                SELECT * 
                FROM transaction_records 
                WHERE id = %s
            """
            db.cursor.execute(query, (transaction_id,))
            return db.cursor.fetchone()

    def update_transaction(
            self, transaction_id, account_name, 
            client_name, transaction_date, 
            amount_receivable, amount_payable,
            settlement_status, notes
        ):
        with Connector() as db:
            query = """
                UPDATE transaction_records 
                SET 
                    account_name = %s, 
                    client_name = %s, 
                    transaction_date = %s,
                    amount_receivable = %s, 
                    amount_payable = %s, 
                    settlement_status = %s, 
                    notes = %s 
                WHERE id = %s
            """
            db.cursor.execute(
                query, 
                (account_name, client_name, 
                transaction_date, amount_receivable,
                amount_payable, settlement_status, 
                notes, transaction_id)
            )
            db.connection.commit()

    def delete_transaction(self, transaction_id):
        with Connector() as db:
            query = """
                DELETE 
                FROM transaction_records 
                WHERE id = %s
            """
            db.cursor.execute(query, (transaction_id,))
            db.connection.commit()

# cashflow_manager.py
from connector import Connector

class CashflowManager:
    def create_cashflow(
            self, transaction_id, dw_date, deposit_amount, 
            withdrawal_amount, notes
        ):
        with Connector() as db:
            query = """
                INSERT INTO cashflow_records 
                    (transaction_id, dw_date, 
                    deposit_amount, 
                    withdrawal_amount, notes) 
                VALUES (%s, %s, %s, %s, %s)
            """
            db.cursor.execute(
                query, 
                (transaction_id, dw_date, deposit_amount, 
                withdrawal_amount, notes)
            )
            db.connection.commit()
            return db.cursor.lastrowid

    def read_cashflow(self, cashflow_id):
        with Connector() as db:
            query = """
                SELECT * 
                FROM cashflow_records 
                WHERE id = %s
            """
            db.cursor.execute(query, (cashflow_id,))
            return db.cursor.fetchone()

    def update_cashflow(
            self, cashflow_id, transaction_id, 
            dw_date, deposit_amount,
            withdrawal_amount, notes
        ):
        with Connector() as db:
            query = """
                UPDATE cashflow_records 
                SET 
                    transaction_id = %s, 
                    dw_date = %s, 
                    deposit_amount = %s,
                    withdrawal_amount = %s, 
                    notes = %s 
                WHERE id = %s
            """
            db.cursor.execute(
                query, 
                (transaction_id, dw_date, deposit_amount, 
                withdrawal_amount, notes, cashflow_id)
            )
            db.connection.commit()

    def delete_cashflow(self, cashflow_id):
        with Connector() as db:
            query = """
                DELETE FROM cashflow_records 
                WHERE id = %s
            """
            db.cursor.execute(query, (cashflow_id,))
            db.connection.commit()

```

### 1-4. 코드 수정 반영
* 함수 이름 축약
* read_all 함수 작성

In [2]:
# transaction_manager
from connector import Connector

class TransactionManager:
    def create(
            self, account_name, client_name, transaction_date, 
            amount_receivable, amount_payable, settlement_status, 
            notes
        ):
        with Connector() as db:
            query = """
                INSERT INTO transaction_records 
                    (account_name, client_name, transaction_date, 
                    amount_receivable, amount_payable, 
                    settlement_status, notes) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            db.cursor.execute(
                query, 
                (account_name, client_name, transaction_date, 
                    amount_receivable, amount_payable, 
                    settlement_status, notes
                )
            )
            db.connection.commit()
            return db.cursor.lastrowid

    def read_all(self):
        with Connector() as db:
            query = "SELECT * FROM transaction_records"
            db.cursor.execute(query)
            return db.cursor.fetchall()

    def read(self, transaction_id):
        with Connector() as db:
            query = "SELECT * FROM transaction_records WHERE id = %s"
            db.cursor.execute(query, (transaction_id,))
            return db.cursor.fetchone()

    def update(
            self, transaction_id, account_name, client_name, 
            transaction_date, amount_receivable, 
            amount_payable, settlement_status, notes
        ):
        with Connector() as db:
            query = """
                UPDATE transaction_records 
                SET account_name = %s, client_name = %s, 
                    transaction_date = %s, amount_receivable = %s,
                    amount_payable = %s, settlement_status = %s,
                    notes = %s 
                WHERE id = %s
            """
            db.cursor.execute(
                query, 
                (account_name, client_name, transaction_date, 
                    amount_receivable, amount_payable, 
                    settlement_status, notes, transaction_id
                )
            )
            db.connection.commit()

    def delete(self, transaction_id):
        with Connector() as db:
            query = "DELETE FROM transaction_records WHERE id = %s"
            db.cursor.execute(query, (transaction_id,))
            db.connection.commit()

# cashflow_manager
from connector import Connector

class CashflowManager:
    def create(
            self, transaction_id, dw_date, deposit_amount, 
            withdrawal_amount, notes
        ):
        with Connector() as db:
            query = """
                INSERT INTO cashflow_records 
                    (transaction_id, dw_date, deposit_amount, 
                    withdrawal_amount, notes) 
                VALUES (%s, %s, %s, %s, %s)
            """
            db.cursor.execute(
                query, 
                (transaction_id, dw_date, deposit_amount, 
                withdrawal_amount, notes)
            )
            db.connection.commit()
            return db.cursor.lastrowid

    def read_all(self):
        with Connector() as db:
            query = "SELECT * FROM cashflow_records"
            db.cursor.execute(query)
            return db.cursor.fetchall()

    def read(self, cashflow_id):
        with Connector() as db:
            query = "SELECT * FROM cashflow_records WHERE id = %s"
            db.cursor.execute(query, (cashflow_id,))
            return db.cursor.fetchone()

    def update(
            self, cashflow_id, transaction_id, dw_date, 
            deposit_amount, withdrawal_amount, notes
        ):
        with Connector() as db:
            query = """
                UPDATE cashflow_records 
                SET transaction_id = %s, dw_date = %s, 
                    deposit_amount = %s, withdrawal_amount = %s, 
                    notes = %s 
                WHERE id = %s
            """
            db.cursor.execute(
                query, 
                (transaction_id, dw_date, deposit_amount, 
                withdrawal_amount, notes, cashflow_id)
            )
            db.connection.commit()

    def delete(self, cashflow_id):
        with Connector() as db:
            query = "DELETE FROM cashflow_records WHERE id = %s"
            db.cursor.execute(query, (cashflow_id,))
            db.connection.commit()


## 2. CRUD 기능을 이용하여 데이터 입력 및 출력 하기
### 2-1. 클래스 인스턴스 선언

In [3]:
tm = TransactionManager()
cm = CashflowManager()

### 2-2. 샘플 데이터 생성

In [6]:
tm.create(
    account_name = "외상매출금", 
    client_name = "ABC전자", 
    transaction_date = "2024-01-10", 
    amount_receivable = 1_000_000, 
    amount_payable = 0, 
    settlement_status = None, 
    notes = None
)

1

In [9]:
tm.read_all()

[{'id': 1,
  'account_name': '외상매출금',
  'client_name': 'ABC전자',
  'transaction_date': datetime.date(2024, 1, 10),
  'amount_receivable': 1000000,
  'amount_payable': 0,
  'settlement_status': None,
  'notes': None}]

In [10]:
tm.create(
    account_name = "외상매출금", 
    client_name = "AG상사", 
    transaction_date = "2024-01-10", 
    amount_receivable = 2_000_000, 
    amount_payable = 0, 
    settlement_status = None, 
    notes = None
)

2

In [11]:
tm.read_all()

[{'id': 1,
  'account_name': '외상매출금',
  'client_name': 'ABC전자',
  'transaction_date': datetime.date(2024, 1, 10),
  'amount_receivable': 1000000,
  'amount_payable': 0,
  'settlement_status': None,
  'notes': None},
 {'id': 2,
  'account_name': '외상매출금',
  'client_name': 'AG상사',
  'transaction_date': datetime.date(2024, 1, 10),
  'amount_receivable': 2000000,
  'amount_payable': 0,
  'settlement_status': None,
  'notes': None}]

### 2-3. 데이터 조회 및 DataFrame으로 변환

In [16]:
import pandas as pd
tmdf = pd.DataFrame(tm.read_all())

In [5]:
tmdf

,id,account_name,client_name,transaction_date,amount_receivable,amount_payable,settlement_status,notes
0,1,외상매출금,ABC전자,2024-01-10,1000000,0,None,None
1,2,외상매출금,AG상사,2024-01-10,2000000,0,None,None


### 2-4. Cashflow 데이터 입력

In [30]:
cm.create(
    transaction_id = 1, 
    dw_date = "2024-01-20", 
    deposit_amount = 500_000, 
    withdrawal_amount = 0, 
    notes = None
)
cm.create(
    transaction_id = 1, 
    dw_date = "2024-01-30", 
    deposit_amount = 500_000, 
    withdrawal_amount = 0, 
    notes = None
)

2

In [6]:
cm.read_all()

[{'id': 1,
  'transaction_id': 1,
  'dw_date': datetime.date(2024, 1, 20),
  'deposit_amount': 500000,
  'withdrawal_amount': 0,
  'notes': None},
 {'id': 2,
  'transaction_id': 1,
  'dw_date': datetime.date(2024, 1, 30),
  'deposit_amount': 500000,
  'withdrawal_amount': 0,
  'notes': None},
 {'id': 3,
  'transaction_id': 2,
  'dw_date': datetime.date(2024, 1, 15),
  'deposit_amount': 1000000,
  'withdrawal_amount': 0,
  'notes': None},
 {'id': 4,
  'transaction_id': 2,
  'dw_date': datetime.date(2024, 1, 25),
  'deposit_amount': 500000,
  'withdrawal_amount': 0,
  'notes': None}]

In [7]:
cm.create(
    transaction_id = 2, 
    dw_date = "2024-01-15", 
    deposit_amount = 1_000_000, 
    withdrawal_amount = 0, 
    notes = None
)
cm.create(
    transaction_id = 2, 
    dw_date = "2024-01-25", 
    deposit_amount = 500_000, 
    withdrawal_amount = 0, 
    notes = None
)

6

In [17]:
cmdf = pd.DataFrame(cm.read_all())

In [18]:
cmdf

,id,transaction_id,dw_date,deposit_amount,withdrawal_amount,notes
0,1,1,2024-01-20,500000,0,None
1,2,1,2024-01-30,500000,0,None
2,3,2,2024-01-15,1000000,0,None
3,4,2,2024-01-25,500000,0,None


### 2-5. pending_account 모듈 작성

* pending_account.py 파일 생성
* TranasctionManager 클래스와 CashflowManager 클래스 복사

In [22]:
from pending_account import (
    TransactionManager as TM, 
    CashflowManager as CM
)

tm = TM()
cm = CM()

In [23]:
tm.read_all()

[{'id': 1,
  'account_name': '외상매출금',
  'client_name': 'ABC전자',
  'transaction_date': datetime.date(2024, 1, 10),
  'amount_receivable': 1000000,
  'amount_payable': 0,
  'settlement_status': None,
  'notes': None},
 {'id': 2,
  'account_name': '외상매출금',
  'client_name': 'AG상사',
  'transaction_date': datetime.date(2024, 1, 10),
  'amount_receivable': 2000000,
  'amount_payable': 0,
  'settlement_status': None,
  'notes': None}]

In [24]:
cm.read_all()

[{'id': 1,
  'transaction_id': 1,
  'dw_date': datetime.date(2024, 1, 20),
  'deposit_amount': 500000,
  'withdrawal_amount': 0,
  'notes': None},
 {'id': 2,
  'transaction_id': 1,
  'dw_date': datetime.date(2024, 1, 30),
  'deposit_amount': 500000,
  'withdrawal_amount': 0,
  'notes': None},
 {'id': 3,
  'transaction_id': 2,
  'dw_date': datetime.date(2024, 1, 15),
  'deposit_amount': 1000000,
  'withdrawal_amount': 0,
  'notes': None},
 {'id': 4,
  'transaction_id': 2,
  'dw_date': datetime.date(2024, 1, 25),
  'deposit_amount': 500000,
  'withdrawal_amount': 0,
  'notes': None}]